# Testing homemade layers

This notebook contains tests for all the homemade layers. The tests takes some samples from the mni-
st dataset as a minibatch and compares the results from running this minibatch through the test CNN
to running the same minibatch through the homemade layers.

In [1]:
#importing relevant libraries

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import os

%cd ..

from CNN_small_architecture import CNNSmall
from Tests.helper_functions import tokenize

/home/emil/Documents/bachelor/bachelorprojekt/Python


In [2]:
MNIST_test = datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]][:1000]

batch_size = 1
test_loader = DataLoader(test_set, batch_size=batch_size)

In [3]:
model_original = CNNSmall()
model_original.eval()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

preds = []

In [5]:
with torch.no_grad():
    for data in test_loader:
        image, label = data
        output = model_original(image)
        if torch.argmax(output) == torch.argmax(label):
            preds.append(1)
        else:
            preds.append(0)
        

In [6]:
preds.count(1)/len(preds)

0.994